In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import numba as nb

def animate_simulation(times, xst, x_range=[-3.0, 6.0], y_range=[0, 1.5], bins=300, x_label='x', y_label='P(x,t)', show_x_eq_distrib=True):
    xx=np.linspace(*x_range, 1000)
    histos=[np.histogram(xst[:,ti], density=True, range=x_range, bins=bins) for ti in range(0,len(times))]
    b=[np.exp(-0.5*k(t)*(xx-center(t))**2)/np.sqrt(2*np.pi/k(t)) for t in times]
    # make figure
    fig_dict = {
        "data": [],
        "layout": {},
        "frames": []
    }
    fig_dict["layout"] = go.Layout(
                            xaxis=dict(range=x_range, autorange=False),
                            yaxis=dict(range=y_range, autorange=False),
                            xaxis_title=x_label,
                            yaxis_title=y_label )
    fig_dict["layout"]["updatemenus"] = [
        {
        "type": "buttons",
        "buttons": [{
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
        }]
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 14},
            "prefix": "t = ",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }
    fig_dict["data"] = [go.Bar(x=histos[0][1], y=histos[0][0], name=y_label)]
    if show_x_eq_distrib:
        fig_dict["data"].append(go.Scatter(x=xx,y=b[0], name = f"Eq. distr."))
    # make frames
    for time_index in range(0, len(times)):
        frame_data = [go.Bar(x=histos[time_index][1], y=histos[time_index][0])]
        if show_x_eq_distrib:
            frame_data.append(go.Scatter(x=xx,y=b[time_index]))
        frame = go.Frame(data=frame_data,
                              name=time_index,
                              traces=[0, 1])
        fig_dict["frames"].append(frame)
        slider_step = {
            "args": [
                [time_index],
                {"frame": {"duration": 300, "redraw": False},
                 "mode": "immediate",
                 "transition": {"duration": 300}}
                ],
                "label": round(times[time_index],3),
                "method": "animate"}
        sliders_dict["steps"].append(slider_step)
    fig_dict["layout"]["sliders"] = [sliders_dict]
    fig=go.Figure(fig_dict)
    fig.update_layout(bargap=0)
    return fig

In [3]:
# Corrección a continuación
@nb.njit
def k3(t):
    """ New ESE protocol by D. Rengifo
    with epsilon != 0 """
    gam=1
    ki=0.5/gam
    kf=1.0/gam
    tf=1.0/(kf*30) # 30 es la proporción entre trelax/tf en el articulo de Trizac
    deltak=kf-ki
    epsilon=(-0.17210447)/tf**4
    s=t/tf
    if t<tf:
        num = ki*(1.0+3.0*deltak*s*(1.0-s)/(tf*ki*kf) - epsilon* (tf**3) * s * (1-3*s+2*s**2) )
        denom = 1 - (deltak/kf)*(3*s**2-2*s**3) + epsilon*(tf**4)*ki*(s**2-2*s**3+s**4)
        return num/denom
    else:
        return kf
@nb.njit
def center3(t):
    return 0.0
@nb.njit
def f3(x,t):
    return -k3(t)*(x-center3(t))
@nb.njit
def U3(x,t):
    return 0.5*k3(t)*(x-center3(t))*(x-center3(t))
@nb.njit
def one_simulation3(dt=0.001, tot_steps=10000, xinit=0.0, noise_scaler=1.0, snapshot_step=100):
    tot_snapshots = int(tot_steps/snapshot_step) + 1
    x = np.zeros(tot_snapshots, dtype=np.float64)
    work = np.zeros_like(x)
    power = np.zeros_like(x)
    heat = np.zeros_like(x)
    delta_U = np.zeros_like(x)
    energy = np.zeros_like(x)
    xold=xinit
    x[0]=xinit
    energy[0] = U3(x[0],0)
    w = 0.0
    q = 0.0
    p = 0.0
    step=0
    snapshot_index=0
    while snapshot_index <= tot_snapshots:
        t=step*dt
        xnew = xold + f3(xold,t)*dt + np.random.normal()*np.sqrt(2.0*dt*noise_scaler)
        p = U3(xnew, t+dt)-U3(xnew,t)
        w = w + p
        q = q + U3(xnew,t)-U3(xold,t)
        step=step+1
        if step % snapshot_step == 0:
            snapshot_index = snapshot_index + 1
            x[snapshot_index] = xnew
            power[snapshot_index] = p/dt
            work[snapshot_index] = w
            heat[snapshot_index] = q
            delta_U[snapshot_index] = U3(xnew,t+dt)-U3(x[0],0)
            energy[snapshot_index] = U3(xnew,t+dt)
        xold=xnew
    return x, power, work, heat, delta_U, energy
@nb.jit(parallel=True)
def many_sims_parallel3(tot_sims = 1000, dt = 0.001, tot_steps =10000, noise_scaler=1.0, snapshot_step=100):
    tot_snapshots = int(tot_steps/snapshot_step)+1
    x = np.zeros((tot_sims, tot_snapshots))
    work = np.zeros_like(x)
    power = np.zeros_like(x)
    heat = np.zeros_like(x)
    delta_U = np.zeros_like(x)
    energy = np.zeros_like(x)
    times=np.arange(0, (1+tot_steps)*dt, dt*snapshot_step)
    for sim_num in nb.prange(tot_sims):
        # initial position taken from equilibrium distribution at t=0
        xinit = np.random.normal(center3(0.0), scale=np.sqrt(1.0/k3(0.0)))
        x[sim_num], power[sim_num], work[sim_num], heat[sim_num], delta_U[sim_num], energy[sim_num] = one_simulation3(dt=dt, tot_steps=tot_steps, xinit=xinit, noise_scaler=noise_scaler, snapshot_step=snapshot_step)
    return times, x, power, work, heat, delta_U, energy

In [4]:
# Corrección a continuación
@nb.njit
def k2(t):
    """ New ESE protocol by D. Rengifo
    with epsilon = 0 """
    gam=1
    ki=0.5/gam
    kf=1.0/gam
    tf=1.0/(kf*30) # 30 es la proporción entre trelax/tf en el articulo de Trizac
    deltak=kf-ki
    epsilon=0.0
    s=t/tf
    if t<tf:
        num = ki*(1.0+3.0*deltak*s*(1.0-s)/(tf*ki*kf) - epsilon* (tf**3) * s * (1-3*s+2*s**2) )
        denom = 1 - (deltak/kf)*(3*s**2-2*s**3) + epsilon*(tf**4)*ki*(s**2-2*s**3+s**4)
        return num/denom
    else:
        return kf
@nb.njit
def center2(t):
    return 0.0
@nb.njit
def f2(x,t):
    return -k2(t)*(x-center2(t))
@nb.njit
def U2(x,t):
    return 0.5*k2(t)*(x-center2(t))*(x-center2(t))
@nb.njit
def one_simulation2(dt=0.001, tot_steps=10000, xinit=0.0, noise_scaler=1.0, snapshot_step=100):
    tot_snapshots = int(tot_steps/snapshot_step) + 1
    x = np.zeros(tot_snapshots, dtype=np.float64)
    work = np.zeros_like(x)
    power = np.zeros_like(x)
    heat = np.zeros_like(x)
    delta_U = np.zeros_like(x)
    energy = np.zeros_like(x)
    xold=xinit
    x[0]=xinit
    energy[0] = U2(x[0],0)
    w = 0.0
    q = 0.0
    p = 0.0
    step=0
    snapshot_index=0
    while snapshot_index <= tot_snapshots:
        t=step*dt
        xnew = xold + f2(xold,t)*dt + np.random.normal()*np.sqrt(2.0*dt*noise_scaler)
        p = U2(xnew, t+dt)-U2(xnew,t)
        w = w + p
        q = q + U2(xnew,t)-U2(xold,t)
        step=step+1
        if step % snapshot_step == 0:
            snapshot_index = snapshot_index + 1
            x[snapshot_index] = xnew
            power[snapshot_index] = p/dt
            work[snapshot_index] = w
            heat[snapshot_index] = q
            delta_U[snapshot_index] = U2(xnew,t+dt)-U2(x[0],0)
            energy[snapshot_index] = U2(xnew,t+dt)
        xold=xnew
    return x, power, work, heat, delta_U, energy
@nb.jit(parallel=True)
def many_sims_parallel2(tot_sims = 1000, dt = 0.001, tot_steps =10000, noise_scaler=1.0, snapshot_step=100):
    tot_snapshots = int(tot_steps/snapshot_step)+1
    x = np.zeros((tot_sims, tot_snapshots))
    work = np.zeros_like(x)
    power = np.zeros_like(x)
    heat = np.zeros_like(x)
    delta_U = np.zeros_like(x)
    energy = np.zeros_like(x)
    times=np.arange(0, (1+tot_steps)*dt, dt*snapshot_step)
    for sim_num in nb.prange(tot_sims):
        # initial position taken from equilibrium distribution at t=0
        xinit = np.random.normal(center2(0.0), scale=np.sqrt(1.0/k2(0.0)))
        x[sim_num], power[sim_num], work[sim_num], heat[sim_num], delta_U[sim_num], energy[sim_num] = one_simulation2(dt=dt, tot_steps=tot_steps, xinit=xinit, noise_scaler=noise_scaler, snapshot_step=snapshot_step)
    return times, x, power, work, heat, delta_U, energy

In [5]:
@nb.njit
def k1(t):
    """ Original ESE from Trizac et al """
    gamma=1.0
    ki=0.5/gamma
    kf=1.0/gamma
    deltak=kf-ki
    tf=1.0/(kf*30) # 30 es la proporción entre trelax/tf en el articulo de Trizac
    s=t/tf
    if t<tf:
        return (3.0*deltak*s*(1-s)/tf)/(ki+deltak*(3.0*s**2-2.0*s**3)) + ki + deltak*(3.0*s**2-2.0*s**3)
    else:
        return kf
@nb.njit
def center1(t):
    return 0.0
@nb.njit
def f1(x,t):
    return -k1(t)*(x-center1(t))
@nb.njit
def U1(x,t):
    return 0.5*k1(t)*(x-center1(t))*(x-center1(t))
@nb.njit
def one_simulation1(dt=0.001, tot_steps=10000, xinit=0.0, noise_scaler=1.0, snapshot_step=100):
    tot_snapshots = int(tot_steps/snapshot_step) + 1
    x = np.zeros(tot_snapshots, dtype=np.float64)
    work = np.zeros_like(x)
    power = np.zeros_like(x)
    heat = np.zeros_like(x)
    delta_U = np.zeros_like(x)
    energy = np.zeros_like(x)
    xold=xinit
    x[0]=xinit
    energy[0] = U1(x[0],0)
    w = 0.0
    q = 0.0
    p = 0.0
    step=0
    snapshot_index=0
    while snapshot_index <= tot_snapshots:
        t=step*dt
        xnew = xold + f1(xold,t)*dt + np.random.normal()*np.sqrt(2.0*dt*noise_scaler)
        p = U1(xnew, t+dt)-U1(xnew,t)
        w = w + p
        q = q + U1(xnew,t)-U1(xold,t)
        step=step+1
        if step % snapshot_step == 0:
            snapshot_index = snapshot_index + 1
            x[snapshot_index] = xnew
            power[snapshot_index] = p/dt
            work[snapshot_index] = w
            heat[snapshot_index] = q
            delta_U[snapshot_index] = U1(xnew,t+dt)-U1(x[0],0)
            energy[snapshot_index] = U1(xnew,t+dt)
        xold=xnew
    return x, power, work, heat, delta_U, energy
@nb.jit(parallel=True)
def many_sims_parallel1(tot_sims = 1000, dt = 0.001, tot_steps =10000, noise_scaler=1.0, snapshot_step=100):
    tot_snapshots = int(tot_steps/snapshot_step)+1
    x = np.zeros((tot_sims, tot_snapshots))
    work = np.zeros_like(x)
    power = np.zeros_like(x)
    heat = np.zeros_like(x)
    delta_U = np.zeros_like(x)
    energy = np.zeros_like(x)
    times=np.arange(0, (1+tot_steps)*dt, dt*snapshot_step)
    for sim_num in nb.prange(tot_sims):
        # initial position taken from equilibrium distribution at t=0
        xinit = np.random.normal(center1(0.0), scale=np.sqrt(1.0/k1(0.0)))
        x[sim_num], power[sim_num], work[sim_num], heat[sim_num], delta_U[sim_num], energy[sim_num] = one_simulation1(dt=dt, tot_steps=tot_steps, xinit=xinit, noise_scaler=noise_scaler, snapshot_step=snapshot_step)
    return times, x, power, work, heat, delta_U, energy

In [12]:
times1, x1, power1, work1, heat1, delta_U1, energy1 = many_sims_parallel1(tot_sims=1000000, dt=0.00001, tot_steps=10000)
times2, x2, power2, work2, heat2, delta_U2, energy2 = many_sims_parallel2(tot_sims=1000000, dt=0.00001, tot_steps=10000)
times3, x3, power3, work3, heat3, delta_U3, energy3 = many_sims_parallel3(tot_sims=1000000, dt=0.00001, tot_steps=10000)

In [13]:
ks1=np.array([k1(t) for t in times1])
ks2=np.array([k2(t) for t in times2])
ks3=np.array([k3(t) for t in times3])

Como son cada protocolo:

In [14]:
px.line(x=times1, y=[ks1, ks2, ks3])

In [17]:
# variancias de x teóricas :
def varx1(t):
    ki=k1(0.0)
    kf =1.0
    tf=1.0/(kf*30) # 30 es la proporción entre trelax/tf en el articulo de Trizac
    deltak=kf-ki

    if t<tf:
        return 1/(ki+deltak*(3*(t/tf)**2-2*(t/tf)**3))
    else:
        return 1/kf
def varx3(t, eps=0.0):
    gamma=1.0
    ki=0.5/gamma
    kf=1.0/gamma
    deltak=kf-ki
    tf=1.0/(kf*30) # 30 es la proporción entre trelax/tf en el articulo de Trizac
    s=t/tf
    if t<tf:
        return 1/ki - (deltak/(ki*kf))*(3.0*(t/tf)**2-2.0*(t/tf)**3) + eps*(tf**4)*(s**2-2*s**3+s**4)
    else:
        return 1/kf
varxteo1 = np.array([varx1(t) for t in times1])
varxteo2 = np.array([varx3(t, eps=0.0) for t in times2])
varxteo3 = np.array([varx3(t, eps=-0.17210447/0.5**4) for t in times3])

In [18]:
px.line(x=times1, y=[np.var(x1, axis=0), varxteo1, np.var(x2, axis=0), varxteo2, np.var(x3, axis=0), varxteo3])

Trabajo promedio

In [19]:
px.line(x=times1, y=[np.average(work1, axis=0), np.average(work2, axis=0), np.average(work3, axis=0)])

Check Jarsynski relation

In [21]:
px.line(x=times2, y=[np.average(np.exp(-work1),axis=0),np.sqrt(ks1[0]/ks1), np.average(np.exp(-work2),axis=0),np.sqrt(ks2[0]/ks2)])

Por revisar: hay un error en la relación de Jarsynski al final del proceso. ¿Será un error numérico de redondeo o algo más de fondo?